# Scrapping

On doit scrappé : 
- Titre d'emploi, 
- Le nom de la société, 
- La localisation, 
- Type de contrat, 
- Salaire,
- Détail du job,
- Temps de travail, 
- Les critères du candidat: niveau d'études minimum requis, la formation, niveau d'expérience minimum, les langues

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pandas as pd
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import concurrent.futures

In [17]:
def searchJob(jobName, country):
    
    
#     if country == 'France':
#         link = 'https://www.indeed.fr'
#     elif country == 'USA':
#         link ='https://www.indeed.com'
#     else:
#         print("Merci d'entrer dans le deuxième argument 'France' ou 'USA'")
#         return
    
    driver = webdriver.Chrome()
    driver.get('https://www.indeed.fr')
    driver.maximize_window()
    
    searchInputwhat = driver.find_element_by_id('text-input-what')
    searchInputwhat.send_keys(jobName)
    # jobsearch-SerpJobCard
    time.sleep(0.2)
    searchInputwhere = driver.find_element_by_id('text-input-where')
    time.sleep(0.2)
    # driver.find_element_by_id('text-input-where').click()

    while len(searchInputwhere.get_attribute('value')) != 0:
        searchInputwhere.send_keys(Keys.BACKSPACE)

    driver.find_element_by_id('text-input-where').send_keys(country)
    # searchInputwhere.send_keys('France')

    bouton_search = driver.find_element_by_class_name('icl-Button')
    bouton_search.click()
    
    try:
        time.sleep(0.3)
        driver.find_element_by_class_name('popover-x-button-close').click()    
    except:
        pass

    try:
        change_country = driver.find_element_by_xpath("//div[@class='invalid_location']/p/a")
        change_country.click()
    except:
        pass
    
    # Les listes vides pour chaque feature
    getInfo = { 
        'Poste' : [],
        'Entreprise' : [],
        'Pays': [],
        'Ville' : [],
        'Type de contrat' : [],
        'Salaire' : [],
        'Description' : []
        }

    page = True

    while page: 

           # Au cas où y a un pop up
        try:
            time.sleep(0.5)
            driver.find_element_by_class_name('popover-x-button-close').click()    
        except:
            pass


        # Trouver la carte de chaque job
        jobcards = driver.find_elements_by_class_name('jobsearch-SerpJobCard')


        # Function récurrente pour scrapping les données
        for job in jobcards:
            job.click()
            time.sleep(0.2)



            # Les noms des posts proposés
            try:
                getInfo['Poste'].append(job.find_element_by_class_name('title').text.replace('\n',''))
                getInfo['Pays'].append(country)
            except NoSuchElementException:
                getInfo['Poste'].append("None")
            
            
            
            # Les nom d'entreprise
            try:
                getInfo['Entreprise'].append(job.find_element_by_class_name('company').text)
            except NoSuchElementException:
                getInfo['Entreprise'].append("None")

            # Les localisations des entreprises
            try:
                getInfo['Ville'].append(job.find_element_by_class_name('local').text)
            except NoSuchElementException:
                getInfo['Ville'].append("None")

            try:
                complementaire = driver.find_element_by_id('vjs-contenaire')
        #         descrpiton = driver.find_element_by_id('vjs-desc')
            except NoSuchElementException:
                pass

            try:
                contrat = driver.find_element_by_css_selector(".jobMetadataHeader-itemWithIcon-icon-jobs + span").text
                getInfo['Type de contrat'].append(contrat)
            except:
                getInfo['Type de contrat'].append("None")

            try:
                salary = job.find_element_by_class_name(".salaryText").text
                getInfo['Salaire'].append(salary)
            except NoSuchElementException:
                getInfo['Salaire'].append("None")

            try:
                description = driver.find_element_by_id('vjs-desc').text.replace('\n', ' ')
                getInfo['Description'].append(description)
            except NoSuchElementException:
                getInfo['Description'].append("None")   

    
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        try:
            element = WebDriverWait(driver, 10).until(           
            EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Suivant'] | //a[@aria-label='Next']")))
            element.click()  
        except TimeoutException:
            page = False
            
    filename = str.lower(jobName.replace(' ','')+country)+".csv".replace(' ','')
    df = pd.DataFrame(getInfo)
    df.to_csv("/data/"+filename,index = False, sep=';', encoding="utf-8-sig")
    print(filename + "complet")
    driver.quit()
        
# ou une autre façon 
# for i in range(0,500,10):
#     driver.get("https://www.indeed.fr/jobs?q=data+science&l=France&start="+str(i))

In [80]:
# container = driver.find_element_by_id('vjs-container')
# soup = BeautifulSoup(container,'html.parser')
# html = driver.page_source
# soup = BeautifulSoup(html, "html.parser")
# print(soup.prettify())

In [12]:
def forSearchJob(jobName, country):
    
    jobName_no_space = jobName.replace(' ','+')
    
    if country == 'France':
        link = 'https://www.indeed.fr/emplois?q="' + jobName_no_space + '"&l=' + country + '&start='
    elif country == 'USA':
        link = 'https://www.indeed.com/jobs?q="' + jobName_no_space + '"&l=' + country + '&start='
    else:
        print("Merci d'entrer dans le deuxième argument 'France' ou 'USA'")
        return

    
    driver = webdriver.Chrome()
    driver.maximize_window()
    
   
    try:
        time.sleep(0.3)
        driver.find_element_by_class_name('popover-x-button-close').click()    
    except:
        pass
    
    # Les listes vides pour chaque feature
    getInfo = { 
        'Poste' : [],
        'Entreprise' : [],
        'Pays': [],
        'Ville' : [],
        'Type de contrat' : [],
        'Salaire' : [],
        'Description' : []
        }
    
    time.sleep(5)
    for i in range(0,500,10):     
        
        driver.get(link+str(i))

        try:
            time.sleep(0.5)
            driver.find_element_by_class_name('popover-x-button-close').click()    
        except:
            pass
    
        # Trouver la carte de chaque job
        jobcards = driver.find_elements_by_class_name('jobsearch-SerpJobCard')

        # Function récurrente pour scrapping les données
        for job in jobcards:
            job.click()
            time.sleep(0.2)

            # Les noms des posts proposés
            try:
                getInfo['Poste'].append(job.find_element_by_class_name('title').text.replace('\n',''))
                getInfo['Pays'].append(country)
            except NoSuchElementException:
                getInfo['Poste'].append("None")
            
            # Les nom d'entreprise
            try:
                getInfo['Entreprise'].append(job.find_element_by_class_name('company').text)
            except NoSuchElementException:
                getInfo['Entreprise'].append("None")

            # Les localisations des entreprises
            try:
                getInfo['Ville'].append(job.find_element_by_class_name('location').text)
            except NoSuchElementException:
                getInfo['Ville'].append("None")
            
            # Les types de contrat des postes 
            try:
                contrat = driver.find_element_by_css_selector(".jobMetadataHeader-itemWithIcon-icon-jobs + span").text
                getInfo['Type de contrat'].append(contrat)
            except:
                getInfo['Type de contrat'].append("None")

            # Les salaires des postes recrutées
            try:
                salary = job.find_element_by_class_name("salaryText").text
                getInfo['Salaire'].append(salary)
            except NoSuchElementException:
                getInfo['Salaire'].append("None")

            # Les descriptions des postes     
            try:
                element = WebDriverWait(driver, 10).until(           
                EC.presence_of_element_located((By.ID, "vjs-desc")))
                description = element.text.replace('\n', ' ')
                getInfo['Description'].append(description)
            except TimeoutException:
                getInfo['Description'].append("None")


    filename = str.lower(jobName.replace(' ','')+country)+".csv".replace(' ','')
    df = pd.DataFrame(getInfo)
    df.to_csv("/data/"+filename,index = False, sep=';', encoding="utf-8-sig")
    print(filename + "complet")
    driver.quit()

In [1]:
# data = searchJob('Data scientist','France')
# data = forSearchJob('Data scientist','France')

In [3]:
jobs = ['Datascientist','Python','JavaScript','Java','Datascientist','Python','JavaScript','Java']
countrys = ['France','France','France','France', 'USA','USA','USA','USA']
# df  = pd.concat([df1, df2,df3], axis = 0).drop_duplicates()

In [19]:
def main():
    jobs = ['Datascientist','Python','JavaScript','Java','Datascientist','Python','JavaScript','Java']
    countrys = ['France','France','France','France', 'USA','USA','USA','USA']
    with concurrent.futures.ProcessPoolExecutor() as executor:  
        executor.map(searchJob, jobs, countrys)
    #     len(results)

if __name__ == '__main__':
    main()

In [16]:
forSearchJob('js', 'France')

jsfrance.csvcomplet


In [15]:
df = pd.read_csv('pythonusa.csv', sep=';', encoding="utf-8-sig")

In [25]:
df['Description'].str.lower().contains("salaire")

AttributeError: 'Series' object has no attribute 'contains'